Speech to text conversion module using wav2vec 2.0 developed by Facebook AI.
Here we will be using Transformer Architecture Hugging face for the development of Speech Recognition system of Duration more than 15 Minutes.
we start with installing transformer and import Wav2Vec2ForCTC, Wav2Vec2Tokenizer.

In [1]:
!pip install -q transformers
import librosa

#Importing Pytorch
import torch

#Importing Wav2Vec
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

     |████████████████████████████████| 2.6 MB 4.9 MB/s 
     |████████████████████████████████| 3.3 MB 41.1 MB/s 
     |████████████████████████████████| 895 kB 49.7 MB/s 
     |████████████████████████████████| 636 kB 50.7 MB/s 


In [34]:
import IPython.display as display
display.Audio("/content/sample_data/audio/sampleaudio.wav", autoplay=True)

In [2]:
!pip install pydub

Inspite of using  T4 GPU and 25GB RAM, we won't be able to process such a large audio file at one time and session would crash even if audio is greater than 2 Minutes with high RAM as well.
Hence to deal with hardware limitations for developing long speech recognition system, I will split the audio files of duration 1 Minutes each and then feed them to the pre trained model wav2vec2.0 .

we will install pydub for this purpose

In [3]:
from pydub import AudioSegment
import math

class SplitWavAudioMubin():
    def __init__(self, folder, filename):
        self.folder = folder
        self.filename = filename
        self.filepath = folder + '/' + filename
        
        self.audio = AudioSegment.from_wav(self.filepath)
    
    def get_duration(self):
        return self.audio.duration_seconds
    
    def single_split(self, from_min, to_min, split_filename):
        t1 = from_min * 60 * 1000
        t2 = to_min * 60 * 1000
        split_audio = self.audio[t1:t2]
        split_audio.export(self.folder + '/' + split_filename, format="wav")
        
    def multiple_split(self, min_per_split):
        total_mins = math.ceil(self.get_duration() / 60)
        for i in range(0, total_mins, min_per_split):
            split_fn = str(i) + '_' + self.filename
            self.single_split(i, i+min_per_split, split_fn)
            print(str(i) + ' Done')
            if i == total_mins - min_per_split:
                print('All splited successfully')

In [10]:
folder = '/content/sample_data/audio'  
file = 'sampleaudio.wav'
split_wav = SplitWavAudioMubin(folder, file)
split_wav.multiple_split(min_per_split=1)

0 Done
1 Done
2 Done
3 Done
4 Done
5 Done
6 Done
All splited successfully


In [11]:
# Loading the audio file
audio_split_1, rate_1 = librosa.load("/content/sample_data/audio/0_sampleaudio.wav", sr = 16000)
audio_split_2, rate_2 = librosa.load("/content/sample_data/audio/1_sampleaudio.wav", sr = 16000)
audio_split_3, rate_3 = librosa.load("/content/sample_data/audio/2_sampleaudio.wav", sr = 16000)
audio_split_4, rate_4 = librosa.load("/content/sample_data/audio/3_sampleaudio.wav", sr = 16000)
audio_split_5, rate_5 = librosa.load("/content/sample_data/audio/4_sampleaudio.wav", sr = 16000)
audio_split_6, rate_6 = librosa.load("/content/sample_data/audio/5_sampleaudio.wav", sr = 16000)
audio_split_7, rate_7 = librosa.load("/content/sample_data/audio/6_sampleaudio.wav", sr = 16000)

In [12]:
# printing audio 
print(audio_split_1)

[ 0.          0.          0.         ... -0.05880737 -0.0501709
 -0.05291748]


In [13]:
print(audio_split_2)

[-0.06158447 -0.07061768 -0.06921387 ...  0.05548096  0.03015137
  0.00924683]


In [14]:
print(audio_split_3)

[-0.00231934 -0.0168457  -0.03164673 ... -0.00592041 -0.00259399
 -0.00469971]


In [15]:
print(audio_split_4)

[-0.00665283 -0.00643921 -0.00619507 ...  0.09960938  0.09985352
  0.09561157]


In [16]:
print(audio_split_5)

[ 0.09338379  0.09432983  0.09655762 ... -0.00125122 -0.00097656
 -0.00299072]


In [17]:
print(audio_split_6)

[-0.00244141  0.00067139  0.00149536 ...  0.01370239  0.01306152
  0.01296997]


In [18]:
print(audio_split_7)

[ 0.01177979  0.00976562  0.00808716 ... -0.00927734 -0.00939941
 -0.009552  ]


In [19]:
# printing rate
print(rate_1)


16000


In [20]:
# Importing Wav2Vec pretrained model

tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.


/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:419: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Now, taking the input values, passing the audio (array) into tokenizer and we want our tensors in PyTorch format instead of Python integers. return_tensors = “pt” which is nothing more than PyTorch format.

In [21]:
# Taking an input value for 6 different splits

In [22]:
input_values_1 = tokenizer(audio_split_1, return_tensors = "pt").input_values

In [23]:
input_values_2 = tokenizer(audio_split_2, return_tensors = "pt").input_values

In [24]:
input_values_3 = tokenizer(audio_split_3, return_tensors = "pt").input_values

In [25]:
input_values_4 = tokenizer(audio_split_4, return_tensors = "pt").input_values

In [26]:
input_values_5 = tokenizer(audio_split_5, return_tensors = "pt").input_values

In [27]:
input_values_6 = tokenizer(audio_split_6, return_tensors = "pt").input_values

In [28]:
input_values_7 = tokenizer(audio_split_7, return_tensors = "pt").input_values

In [29]:
# Storing logits (non-normalized prediction values)
logits_1 = model(input_values_1).logits

In [34]:
# Storing logits (non-normalized prediction values)
logits_2 = model(input_values_2).logits

In [39]:
# Storing logits (non-normalized prediction values)
logits_3 = model(input_values_3).logits

In [44]:
# Storing logits (non-normalized prediction values)
logits_4 = model(input_values_4).logits

In [49]:
# Storing logits (non-normalized prediction values)
logits_5 = model(input_values_5).logits

In [54]:
# Storing logits (non-normalized prediction values)
logits_6 = model(input_values_6).logits

In [59]:
# Storing logits (non-normalized prediction values)
logits_7 = model(input_values_7).logits

In [30]:
# Storing logits (non-normalized prediction values)
prediction_1 = torch.argmax(logits_1, dim = -1)

In [35]:
# Storing predicted ids
prediction_2 = torch.argmax(logits_2, dim = -1)

In [40]:
# Storing predicted ids
prediction_3 = torch.argmax(logits_3, dim = -1)

In [45]:
# Storing predicted ids
prediction_4 = torch.argmax(logits_4, dim = -1)

In [50]:
# Storing predicted ids
prediction_5 = torch.argmax(logits_5, dim = -1)

In [55]:
# Storing predicted ids
prediction_6 = torch.argmax(logits_6, dim = -1)

In [60]:
# Storing predicted ids
prediction_7 = torch.argmax(logits_7, dim = -1)

In [31]:
# Passing the prediction to the tokenzer decode to get the transcription
transcription_1 = tokenizer.batch_decode(prediction_1)[0]

In [36]:
# Passing the prediction to the tokenzer decode to get the transcription
transcription_2 = tokenizer.batch_decode(prediction_2)[0]

In [41]:
# Passing the prediction to the tokenzer decode to get the transcription
transcription_3 = tokenizer.batch_decode(prediction_3)[0]

In [46]:
# Passing the prediction to the tokenzer decode to get the transcription
transcription_4 = tokenizer.batch_decode(prediction_4)[0]

In [51]:
# Passing the prediction to the tokenzer decode to get the transcription
transcription_5 = tokenizer.batch_decode(prediction_5)[0]

In [56]:
# Passing the prediction to the tokenzer decode to get the transcription
transcription_6 = tokenizer.batch_decode(prediction_6)[0]

In [61]:
# Passing the prediction to the tokenzer decode to get the transcription
transcription_7 = tokenizer.batch_decode(prediction_7)[0]

In [32]:
# Printing the transcription
print(transcription_1)


THERE IS AN INDEFINABLE MYSTERIOUS POWER THAT PERVADES EVERYTHING I FEEL IT THOUGH I DO NOT SEE IT IT IS THIS UNSEEN POWER WHICH MAKES ITSELF FELT AND YET DEFIIS ALL PRUE BECAURSE IT IS SO UNLIKE ALL THAT I PERCEIVE THROUGH MY SENSES IT TRANSCENDS THE SENSES BUT IT IS POSSIBLE TO RISEN OUT THE EXISTENCE OF GOD TO A LIMITED EXTENT EVEN IN ORDINARY AFFAIRS WE KNOW THAT PEOPLE DO NOT KNOW WHO RULES OR WHY AND HOW HE RLE AND YET THEY KNOW THAT THERE IS A POWER THAT CERTAINLY RULE IN MY TOR LAST YEAR IN MY SOUR I MIT MANY POOR VILLAGERS AND I HOUND UPON INQUIRY THAT THEY DID NOT KNOW WHO RULE MY SOLF THEY SIMPLY S


In [58]:
from sklearn.externals import joblib
from google.colab import files

#you can save variable into file on colab files

joblib.dump(logits_6,  'logits_6.pkl')   
 
#this will download file to your local downloads

files.download('logits_6.pkl')       

#reload your saved data.

logits_6 = joblib.load('logits_6.pkl') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [37]:
print(transcription_2)

ID SOME GOD RULED IF THE KNOWLEDGE OF THESE POOR PEOPLE WAS SO LIMITED ABOUT LTHEIR RULER I WHO AM INFINITELY LITTER IN RESPECT TO GOD THEN THEIL DO THEIR RULER NEED NOT BE SURPRISED IF I DO NOT REALIZE THE PRESENCE OF GOD THE KING OF KINGS NEVERTHELESS I DO FEEL AS THE POOR RELITERS FAILT ABOUT MY SWOD THAT THERE IS ORDERLINESS IN THE UNIVERSE THERE IS AN UNALTERABLE LAW GOVERNING EVERYTHING AND EVERY BEING THAT EXISTS OR LIVES IT IS NOT A BLIND LAW FOR NO BLIND LAW CAND GOVERN THE CONDUCT OF LIVING BEINGS AND THANKS TO THE MARVELLOUS RESURCHES OF THAT DEASY BOTH IT CAN NOW BE PROVED THAT EVEN MATTER IS LIFE THAT LAW THEN WHICH GOVERNS ALL LIFE IS GOD LAW AND THE LAWGIVER


In [42]:
print(transcription_3)

AR ONE I MAY NOT DENY THE LAW OR DHE LOGIVER BECAUSE I KNOW SO LITTLE ABOUT IT OR HIM JUST AS MY DENIAL OR IGNORANCE OF THE EXISTENCE OF AN EARTHLY POWER WILL AVAIL ME NOTHING EVEN SO MY DENIAL OF GOD AND HIS LAW WILL NOT ELIBERATE ME FROM ITS OPPARITION WHEREAS HUMBLE AND MUTE ACCEPTANCE OF DIVINE AUTHORITY MAKES LIFE JOURNEY EASIER EVEN AS THE ACCEPTANCE OF EARTHLY RULE MAKES LIFE UNDER IT EASIER I DO DIMLY PERCEIVE THAT WHILE EVERYTHING AROUND ME IS EVER CHANGING EVER DYING THEREIS UNDERLYING ALL THAT CAN IT LVING POWER THAT IT CHANGELESS THAT HOLDS ALL TOGETHER THAT CREAKES TELONG


In [47]:
print(transcription_4)

AND RECRIATE THAT INFORMING POWER OR SPIRIT IS GOD AND SINCE NOTHING ELSE THAT I SEE MERELY THROUGH THE SINSES CAN OR WILL PERSIST HE ALONE IS AND IS THIS POWER BENEVOLENT OR MENEVOLENT I SEE IT AS PURELY BENEVOLENT GOD I CAN SEE THAT IN THE MIDST OF THIS LIFE PERSSE IN THE MIDST OF UNTRUTH TRUTH PERSISEDIN THE MIDST OF DARKNESS LIFE PERSISED HINCE I GATHER THAT GOD IS LIFE TRUTH LIFE HE IS LOVE HE IS THE SUPREME GOOD BUT HE IS NO GOD WHO MERELY SATISFIES THE INTELLECT IF HE EVER DOES GOD TO BE GOD MUST TE RULE THE HEART AND TRANSFORM IT HE MUST EXPRESS HIMSELF IS


In [52]:
print(transcription_5)

N EVERY SMALLEST ACT OF HIS GOODERY THIS CAN ONLY BE DONE THROUGH A DEFINITE REALIZATION MORE REAL THEN THE FIVE SENSES CAN EVER PROVE USE SINCE PERCEPTIONS CAN BE AND OFTEN OUR FAULSE AND DECEPTIVE HOWEVER REAL THEY MAY APPEAR TO US WHERE THERE IS REALIZATION OUTSIDE THE SENSES IT IS INFARIBLE IT IS PROVED NOT BY EXTREME EXTRENIOUS EVIDEN BUT IN THE TRANSFORMED CONDUCT AND CHARACTER OF THOSE WHO HAVE FELT THE REAL PRESENCE OF GOD WITHIN SUCH TESTIMONY IS TO BE FOUND IN THE EXPERIENSES OF AN UNBROKEN LINE OF PROFITS AND SEDIES IN ALL COUNTRIES AND CLIME TO REJECT THIS EVIDENCE IS TO DENY VEN


In [57]:
print(transcription_6)

SELF THIS REALIZATION IS PRECEDED BY AN IMMOVABLE FAITH HE WHO WOULD IN HIS OWN PERSON TS THE FACT OF GOD'S PRISON CAN DO SO BY A LIVING FAITH AND SINCE FASH ITSELF CAN NOT BE PROVED BY EXTRENIOUS EVIDENCE THE FIFEST CAUSE IS TO BELIEVE IN THE MORAL GOVERNMENT OF THE WORLD AND THEREFORE IN THE SUPREMACY OF THE MORAL LAW THE LAW OF TRUTH AND LOVE EXERCISE OF FAIH WILL BE THE FAFAITH WHERE THERE IS I CERE DETERMINATION SUMMARLY TO REDICT ALL THAT IS CONTRARY TO TRUTH AND LOE I CONFESS THAT I HAVE NO ARGUMENT TO CONVINCE THROUGH REASON FIS TRANTINGS REASON ALL AS I CAN ADVISE IS NOT TO ATTEND TO THE IMPOSSIBLE


In [62]:
print(transcription_7)

In [64]:
full_text = transcription_1 + transcription_2 + transcription_3 + transcription_4 + transcription_5 + transcription_6 + transcription_7

In [65]:
print(full_text)

THERE IS AN INDEFINABLE MYSTERIOUS POWER THAT PERVADES EVERYTHING I FEEL IT THOUGH I DO NOT SEE IT IT IS THIS UNSEEN POWER WHICH MAKES ITSELF FELT AND YET DEFIIS ALL PRUE BECAURSE IT IS SO UNLIKE ALL THAT I PERCEIVE THROUGH MY SENSES IT TRANSCENDS THE SENSES BUT IT IS POSSIBLE TO RISEN OUT THE EXISTENCE OF GOD TO A LIMITED EXTENT EVEN IN ORDINARY AFFAIRS WE KNOW THAT PEOPLE DO NOT KNOW WHO RULES OR WHY AND HOW HE RLE AND YET THEY KNOW THAT THERE IS A POWER THAT CERTAINLY RULE IN MY TOR LAST YEAR IN MY SOUR I MIT MANY POOR VILLAGERS AND I HOUND UPON INQUIRY THAT THEY DID NOT KNOW WHO RULE MY SOLF THEY SIMPLY SID SOME GOD RULED IF THE KNOWLEDGE OF THESE POOR PEOPLE WAS SO LIMITED ABOUT LTHEIR RULER I WHO AM INFINITELY LITTER IN RESPECT TO GOD THEN THEIL DO THEIR RULER NEED NOT BE SURPRISED IF I DO NOT REALIZE THE PRESENCE OF GOD THE KING OF KINGS NEVERTHELESS I DO FEEL AS THE POOR RELITERS FAILT ABOUT MY SWOD THAT THERE IS ORDERLINESS IN THE UNIVERSE THERE IS AN UNALTERABLE LAW GOVERNING